In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.initializers import *
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

import time
from sklearn.model_selection import KFold

In [2]:
# Model params
NAME = f"MLP121-at{int(time.time())}"
lr = 0.001
optimizer = Adam(lr=lr)
EPOCHS = 600
BATCH_SIZE = 1054

In [3]:
# Load Data
features = np.loadtxt("./features.txt")

In [4]:
X = np.array([x[0:16] for x in features])
y = np.array([int(x[16:17]-1)for x in features])

In [5]:
# Dataset Variables
num_features = 16
num_classes = 11

In [6]:
def create_model(X_train):
    model = Sequential()

    model.add(Dense(units=15, kernel_initializer="uniform", bias_initializer="zeros", input_shape=X_train.shape[1:], activation="relu"))

    model.add(Dense(units=num_classes, kernel_initializer="uniform", bias_initializer="zeros", activation="softmax"))

    model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"])

    return model

In [7]:
tensorboard = TensorBoard(
    log_dir=f"logs/{NAME}",
    histogram_freq=1,
    write_graph=True
    )

filepath = "MLP-{epoch:02d}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor="val_acc", verbose = 0, save_best_only = True, mode = "max"))

kfold = KFold(n_splits=10, shuffle=True)
fold_no = 1
acc_per_fold = []
loss_per_fold = []

for train, test in kfold.split(X, y):
    # Create model
    model = create_model(X[train])

    print("------------------------------------")
    print(f"Fitting Fold no. {fold_no}")
    model.fit(
        X[train], y[train],
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X[test], y[test]),
        #callbacks=[tensorboard, checkpoint]
    )

    results = model.evaluate(X[test], y[test])
    print(f"Score for {fold_no}: {results}")

    # Push fold loss
    loss_per_fold.append(results[0])

    # Push fold acc
    acc_per_fold.append(results[1])

    fold_no = fold_no + 1

print('--------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('--------------------------------------------------------------')
    print(f"> Fold {i+1} - Accuracy: {acc_per_fold[i]}%")
print('--------------------------------------------------------------')
print("Average scores for all folds:")
print(f"> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})")
print('--------------------------------------------------------------')

------------------------------------
Fitting Fold no. 1
Epoch 1/600
49/49 [==============================] - 0s 4ms/step - loss: 2.1931 - accuracy: 0.1471 - val_loss: 1.9527 - val_accuracy: 0.2121
Epoch 2/600
49/49 [==============================] - 0s 2ms/step - loss: 1.7437 - accuracy: 0.3946 - val_loss: 1.5154 - val_accuracy: 0.5543
Epoch 3/600
49/49 [==============================] - 0s 2ms/step - loss: 1.3411 - accuracy: 0.6409 - val_loss: 1.1802 - val_accuracy: 0.6991
Epoch 4/600
49/49 [==============================] - 0s 2ms/step - loss: 1.1022 - accuracy: 0.6845 - val_loss: 1.0180 - val_accuracy: 0.7016
Epoch 5/600
49/49 [==============================] - 0s 2ms/step - loss: 0.9876 - accuracy: 0.7029 - val_loss: 0.9353 - val_accuracy: 0.7141
Epoch 6/600
49/49 [==============================] - 0s 2ms/step - loss: 0.9181 - accuracy: 0.7178 - val_loss: 0.8764 - val_accuracy: 0.7525
Epoch 7/600
49/49 [==============================] - 0s 2ms/step - loss: 0.8698 - accuracy: 0.7300

49/49 [==============================] - 0s 2ms/step - loss: 0.5064 - accuracy: 0.8476 - val_loss: 0.5009 - val_accuracy: 0.8569
Epoch 59/600
49/49 [==============================] - 0s 2ms/step - loss: 0.5041 - accuracy: 0.8480 - val_loss: 0.5010 - val_accuracy: 0.8534
Epoch 60/600
49/49 [==============================] - 0s 2ms/step - loss: 0.5028 - accuracy: 0.8486 - val_loss: 0.4993 - val_accuracy: 0.8557
Epoch 61/600
49/49 [==============================] - 0s 2ms/step - loss: 0.5013 - accuracy: 0.8486 - val_loss: 0.4979 - val_accuracy: 0.8569
Epoch 62/600
49/49 [==============================] - 0s 2ms/step - loss: 0.5006 - accuracy: 0.8492 - val_loss: 0.4971 - val_accuracy: 0.8565
Epoch 63/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4995 - accuracy: 0.8484 - val_loss: 0.4963 - val_accuracy: 0.8555
Epoch 64/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4989 - accuracy: 0.8492 - val_loss: 0.4964 - val_accuracy: 0.8574
Epoch 65/600
49/49 

Epoch 116/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4579 - accuracy: 0.8580 - val_loss: 0.4598 - val_accuracy: 0.8646
Epoch 117/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4584 - accuracy: 0.8578 - val_loss: 0.4624 - val_accuracy: 0.8647
Epoch 118/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4575 - accuracy: 0.8588 - val_loss: 0.4600 - val_accuracy: 0.8626
Epoch 119/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4574 - accuracy: 0.8582 - val_loss: 0.4596 - val_accuracy: 0.8628
Epoch 120/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4561 - accuracy: 0.8579 - val_loss: 0.4590 - val_accuracy: 0.8661
Epoch 121/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4557 - accuracy: 0.8583 - val_loss: 0.4576 - val_accuracy: 0.8661
Epoch 122/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4554 - accuracy: 0.8583 - val_loss: 0.4607 - val_accuracy: 0.8614

49/49 [==============================] - 0s 2ms/step - loss: 0.4324 - accuracy: 0.8650 - val_loss: 0.4390 - val_accuracy: 0.8687
Epoch 174/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4321 - accuracy: 0.8651 - val_loss: 0.4367 - val_accuracy: 0.8712
Epoch 175/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8645 - val_loss: 0.4377 - val_accuracy: 0.8686
Epoch 176/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4314 - accuracy: 0.8656 - val_loss: 0.4361 - val_accuracy: 0.8700
Epoch 177/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4308 - accuracy: 0.8651 - val_loss: 0.4360 - val_accuracy: 0.8696
Epoch 178/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4309 - accuracy: 0.8654 - val_loss: 0.4372 - val_accuracy: 0.8705
Epoch 179/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4306 - accuracy: 0.8647 - val_loss: 0.4354 - val_accuracy: 0.8714
Epoch 180/600

49/49 [==============================] - 0s 2ms/step - loss: 0.4151 - accuracy: 0.8692 - val_loss: 0.4198 - val_accuracy: 0.8743
Epoch 231/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4143 - accuracy: 0.8708 - val_loss: 0.4192 - val_accuracy: 0.8748
Epoch 232/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4134 - accuracy: 0.8702 - val_loss: 0.4211 - val_accuracy: 0.8715
Epoch 233/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4132 - accuracy: 0.8700 - val_loss: 0.4202 - val_accuracy: 0.8719
Epoch 234/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4135 - accuracy: 0.8703 - val_loss: 0.4217 - val_accuracy: 0.8719
Epoch 235/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4126 - accuracy: 0.8702 - val_loss: 0.4191 - val_accuracy: 0.8731
Epoch 236/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4140 - accuracy: 0.8689 - val_loss: 0.4182 - val_accuracy: 0.8733
Epoch 237/600

...

49/49 [==============================] - 0s 2ms/step - loss: 0.3610 - accuracy: 0.8791 - val_loss: 0.3491 - val_accuracy: 0.8844
Epoch 486/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3609 - accuracy: 0.8793 - val_loss: 0.3495 - val_accuracy: 0.8818
Epoch 487/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3607 - accuracy: 0.8791 - val_loss: 0.3483 - val_accuracy: 0.8820
Epoch 488/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3610 - accuracy: 0.8790 - val_loss: 0.3472 - val_accuracy: 0.8832
Epoch 489/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3605 - accuracy: 0.8790 - val_loss: 0.3517 - val_accuracy: 0.8813
Epoch 490/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3606 - accuracy: 0.8791 - val_loss: 0.3517 - val_accuracy: 0.8813
Epoch 491/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3606 - accuracy: 0.8787 - val_loss: 0.3486 - val_accuracy: 0.8804
Epoch 492/600

49/49 [==============================] - 0s 2ms/step - loss: 0.3573 - accuracy: 0.8801 - val_loss: 0.3483 - val_accuracy: 0.8822
Epoch 543/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3575 - accuracy: 0.8799 - val_loss: 0.3492 - val_accuracy: 0.8808
Epoch 544/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3571 - accuracy: 0.8803 - val_loss: 0.3489 - val_accuracy: 0.8846
Epoch 545/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3567 - accuracy: 0.8803 - val_loss: 0.3498 - val_accuracy: 0.8851
Epoch 546/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3575 - accuracy: 0.8800 - val_loss: 0.3440 - val_accuracy: 0.8850
Epoch 547/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3572 - accuracy: 0.8802 - val_loss: 0.3436 - val_accuracy: 0.8848
Epoch 548/600
49/49 [==============================] - 0s 2ms/step - loss: 0.3562 - accuracy: 0.8806 - val_loss: 0.3445 - val_accuracy: 0.8832
Epoch 549/600

49/49 [==============================] - 0s 2ms/step - loss: 0.3534 - accuracy: 0.8809 - val_loss: 0.3421 - val_accuracy: 0.8857
Epoch 600/600
180/180 [==============================] - 0s 652us/step - loss: 0.3454 - accuracy: 0.8850
Score for 2: [0.34543463587760925, 0.8849573135375977]
------------------------------------
Fitting Fold no. 3
Epoch 1/600
49/49 [==============================] - 0s 3ms/step - loss: 1.6178 - accuracy: 0.4250 - val_loss: 1.0845 - val_accuracy: 0.6639
Epoch 2/600
49/49 [==============================] - 0s 2ms/step - loss: 0.9644 - accuracy: 0.6920 - val_loss: 0.8958 - val_accuracy: 0.7091
Epoch 3/600
49/49 [==============================] - 0s 2ms/step - loss: 0.8360 - accuracy: 0.7363 - val_loss: 0.8006 - val_accuracy: 0.7614
Epoch 4/600
49/49 [==============================] - 0s 2ms/step - loss: 0.7659 - accuracy: 0.7697 - val_loss: 0.7406 - val_accuracy: 0.7880
Epoch 5/600
49/49 [==============================] - 0s 2ms/step - loss: 0.7157 - accuracy:

49/49 [==============================] - 0s 2ms/step - loss: 0.4773 - accuracy: 0.8574 - val_loss: 0.4904 - val_accuracy: 0.8545
Epoch 56/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4762 - accuracy: 0.8570 - val_loss: 0.4885 - val_accuracy: 0.8550
Epoch 57/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4758 - accuracy: 0.8559 - val_loss: 0.4889 - val_accuracy: 0.8550
Epoch 58/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.8568 - val_loss: 0.4884 - val_accuracy: 0.8558
Epoch 59/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4737 - accuracy: 0.8579 - val_loss: 0.4843 - val_accuracy: 0.8576
Epoch 60/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4721 - accuracy: 0.8579 - val_loss: 0.4861 - val_accuracy: 0.8558
Epoch 61/600
49/49 [==============================] - 0s 2ms/step - loss: 0.4711 - accuracy: 0.8586 - val_loss: 0.4831 - val_accuracy: 0.8567
Epoch 62/600
49/49 

KeyboardInterrupt: 